# Encode and Obbfuscate

In [8]:
import heapq
from collections import Counter
import numpy as np  

def build_huffman_tree_Optimized(text, base=2):
    """
    Build a Huffman tree for different base encodings, translating paths to DNA bases
    or corresponding symbols immediately to avoid ambiguity in path representation.
    """
    assert base >= 2 and base <= 22 # Ensure valid base

    # Mapping for DNA bases and extended symbols for bases 11 and 12
    base_to_dna ={0:'W', 1:'S', 2:'M', 3:'K', 4:'Y', 5:'R', 6:'T', 7:'C', 8:'G', 9:'A'}
    # Count frequency of each character in the text
    frequency = Counter(text)
    N=1
    while (base-1)*N+1 < len(frequency):
       N+=1
    add_dum=(base-1)*N+1-len(frequency) 
    # add add_dum dummy characters to the frequency, with 0 frequency
    for i in range(add_dum):
        dummy_char ="X"+str(i)
        frequency[dummy_char] = 0
    
    # Initialize priority queue with frequency count
    heap = [[weight, [symbol, ""]] for symbol, weight in frequency.items()]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        combined_weight = 0
        new_node = []
        
        # Combine up to 'base' nodes from the heap
        for i in range(min(base, len(heap))):
            
            node = heapq.heappop(heap)
            
            combined_weight += node[0]
         
                

            # Append a unique path digit for each combined node, directly translating to DNA bases
            for symbol, path in node[1:]:
    
                new_path = base_to_dna[i] + path  # Translate immediately to DNA base
                new_node.append([symbol, new_path])
                
            
        heapq.heappush(heap, [combined_weight] + new_node)
    return sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[-1]), p))



def huffman_encoding(text, huffman_tree):
    """
    Encode the text using the Huffman tree, translating the path steps directly to encoded characters.
    """
    # Map symbols to their codes from the Huffman tree
    huffman_dict = {symbol: code for symbol, code in huffman_tree}
    # Encode the text using the Huffman codes
    encoded_text = ''.join(huffman_dict[char] for char in text)
    
    return encoded_text

def huffman_decoding(encoded_text, huffman_tree):
    """
    Decode the text using the Huffman tree, translating encoded characters back to the original symbols.
    """
    # Reverse the Huffman dictionary to map codes to symbols
    huffman_dict = {code: symbol for symbol, code in huffman_tree}
    
    decoded_text = ""
    current_code = ""
    for digit in encoded_text:
        current_code += digit
        if current_code in huffman_dict:
            decoded_text += huffman_dict[current_code]
            current_code = ""  # Reset for the next symbol
    
    return decoded_text



In [9]:
import random
import numpy as np
def decompsiotion(sequence):
    combined_bases = {'R': ['A', 'G'], 'Y': ['C', 'T'], 'K': ['G', 'T'], 'M': ['A', 'C'], 'S': ['C', 'G'], 'W': ['A', 'T']}
   # Replace combined bases with proper bases
    expanded_sequence_1= ''
    expanded_sequence_2= ''
    for base in sequence:
        if base in combined_bases:
            choice_1 = random.choice(combined_bases[base])
            expanded_sequence_1 += choice_1

            # For sequence_2, choose the other base
            remaining_choices = [b for b in combined_bases[base] if b != choice_1]
            choice_2 = remaining_choices[0]  # Since there is always exactly one other choice
            expanded_sequence_2 += choice_2
        else:
            expanded_sequence_1 += base
            expanded_sequence_2 += base
    return expanded_sequence_1, expanded_sequence_2
    


def simulate_reads(sequence, error_rate, num_reads):
  
  # Set the seed to the current time

  results = []

  # Replace combined bases with proper bases
  for num in range(num_reads):
  #simulate erros
    read = ''
    for base in sequence:
      if random.random() < error_rate:
        read += np.random.choice(['A', 'C', 'G', 'T'])
      else:
        read += base

    results.append(read)
  return results


from collections import defaultdict, Counter
import numpy as np

def read_fasta(filename):
    """Reads sequences from a FASTA file."""
    sequences = []
    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequences.append(line.strip())
    return sequences

def calculate_base_frequencies(sequences):
    """Calculates the frequency of each base at every position."""
    sequence_length = len(sequences[0])
    base_counts = [Counter() for _ in range(sequence_length)]

    for seq in sequences:
        for i, base in enumerate(seq):
            base_counts[i][base] += 1

    base_frequencies = []
    for counts in base_counts:
        total = sum(counts.values())
        frequencies = {base: count / total for base, count in counts.items()}
        base_frequencies.append(frequencies)
    return base_frequencies

def apply_cutoff(base_frequencies, cutoff=0.05):
    """Applies a frequency cutoff to base frequencies."""
    return [{base: freq for base, freq in frequencies.items() if freq >= cutoff} for frequencies in base_frequencies]

def replace_with_combined_bases(frequencies):
    """Replaces bases with IUPAC combined bases according to the frequencies."""
    iupac_codes = {
        frozenset(['A', 'G']): 'R', frozenset(['C', 'T']): 'Y',
        frozenset(['G', 'T']): 'K', frozenset(['A', 'C']): 'M',
        frozenset(['G', 'C']): 'S', frozenset(['A', 'T']): 'W',
        frozenset(['A']): 'A', frozenset(['C']): 'C',
        frozenset(['G']): 'G', frozenset(['T']): 'T',
    }

    consensus_sequence = ''
    for pos_freq in frequencies:
        significant_bases = frozenset(pos_freq.keys())
        consensus_sequence += iupac_codes.get(significant_bases, 'A')  # 'N' if no significant bases or combination not in table

    return consensus_sequence

In [10]:
import random
import numpy as np
def obfuscation_data_generate(Data):
    combined_bases = {'R': ['A', 'G'], 'Y': ['C', 'T'], 'K': ['G', 'T'], 'M': ['A', 'C'], 'S': ['C', 'G'], 'W': ['A', 'T']}
    obfuscation=[]
    for i in range(len(Data)):
        if Data[i] in ['A','T','C','G']:
            # replace the base different from the original base
            obfuscation.append(np.random.choice([base for base in ['A','T','C','G'] if base!=Data[i]]))
        else:
          # add the the base the same as original base
            obfuscation.append(Data[i])
    obfuscation=''.join(obfuscation)
    return obfuscation

In [11]:
def decryption  (consensus_sequence,obfuscation_seqs):
    combined_bases = {'R': ['A', 'G'], 'Y': ['C', 'T'], 'K': ['G', 'T'], 'M': ['A', 'C'], 'S': ['C', 'G'], 'W': ['A', 'T']}
    decrypted=''
    for i in range(len(consensus_sequence)):
        if consensus_sequence[i] == obfuscation_seqs[i]:
            decrypted+=consensus_sequence[i]
        else:
            choice=[b for b in combined_bases[consensus_sequence[i]] if b != obfuscation_seqs[i]]
            decrypted+=choice[0]
    return decrypted

In [12]:
example_text="""When I find myself in times of trouble
Mother Mary comes to me
Speaking words of wisdom
Let it be
And in my hour of darkness
She is standing right in front of me
Speaking words of wisdom
Let it be
Let it be, let it be
Let it be, let it be
Whisper words of wisdom
Let it be
"""


In [13]:
huffman_tree = build_huffman_tree_Optimized(example_text, base=10)
encoded_text=huffman_encoding(example_text, huffman_tree)
print(len(example_text))
print(len(encoded_text))
print(encoded_text)

273
376
TMAKCSGTRGGATYSAGGACTGMCAWATGYSGRYACCMGKATGRAAKTKARAWCWTSKRAKCAAGTSAMAATGGTRAKACCMGRKGACCWTYTCCAMTTYSTAGAYKAAAGMGKATGAYYMAGKACWASCRGYRGARCWTRCSAGGYSGACTGGAKKTKAAGKATGAGAMAATTSCMMWTYAKCGYMGMRAMSAGYSTAGAAYTAAKRGYSGATAAKSRGKATGACCWTYTCCAMTTYSTAGAYKAAAGMGKATGAYYMAGKACWASCRGYRGARCWASCRGYRGARCTWGAWCRGYRGARCWASCRGYRGARCTWGAWCRGYRGARCWTMAKYMTCCAAGAYKAAAGMGKATGAYYMAGKACWASCRGYRGARCW


In [14]:
obfu_1="GACGACTCGACCATCCACTGGTATCTTGACCCACGGTCAGGGAGGCGAGTGTTATGGTGCTAGCCGGCAGGGCACAGTCATCTAGCGATTCCATTGCCCCCCGCTTGTTGCAATCAACTTAGCGCGTGCAATTGACATTAGGGGTTTCCGACTTTGTGTGTTTGCATATCCTAACACATATGGATCCCCATACTCTCGCCCTCGCCGGATGTTCCCTCGTGTACTGGAATCGATCCACGCCCCCTTGTACAGCAAGCTCGATTAACGAAATACCATTGCGAATGTGGAGTTGTTATCATCGGAGCTGCCGTCAACATCAAATTATTGTAGACGTCATATTAGTTTTTTCAGCGCTCTCTAGGAATCAACTAAGGAA"
obfu_2="GCCTAGTCAACCACGCACTGGTCTCATGATGCGTGGTAATGGAAGCTATTATAAAGCGACGAGCCGCCCGGGCACGGGCATATGTCGATACTATTGACCTGCGCTCTTTGCCAGCAACCCCGCTCGAGGAGTCAACGTAAAGCGTTCGCGACTTTTGGGGTTGGCATATACTAAGAACAACGTATTACAGTCGTCCGGCCCTTGCCTAACCTTCCCGGATTTACTGGTACCGATACATCCCCCTGTGTAAATCAAGTCAGAGTATCCAGACGCCGTAGGGGACATGAAGATGATGTTGTCAGTGGTACTATCGACTTCTAGTCGTTATTGCCTCAATATTAGCGTTTTAATCGCTTCATATGATTGAGCCGAGAAT"
encode_1="TAATCCGTGGGATCCAGGACTGACAAATGCCGGTACCCGTATGAAAGTGAAATCATGTAAGCAAGTGAAAATGGTAAGACCCGATGACCATTTCCAATTTCTAGACTAAAGAGGATGACTAAGTACAAGCAGCGGAGCATGCCAGGCGGACTGGATTTTAAGGATGAGACAATTGCCCTTTAGCGCCGAGAACAGTCTAGAATTAATGGTCGATAAGCAGTATGACCATTTCCACTTCCTAGATGAAAGCGTATGACTAAGTACTACCGGTGGAACTACCAGTGGAACTAGAACAGTAGAACTACCAGCGGAACTTGATCAGCAGAGCATAATCCTCCAAGATTAAAGCGGATGACCAAGTACAAGCAGCAGAACT"
encode_2="TCAGCGGTAGGATTGAGGACTGCCATATGTGGACACCAGGATGGAATTTAGAACTTCGGATCAAGTCACAATGGTGATACCAGGGGACCTTCTCCACTTCGTAGATGAAAGCGTATGATCCAGGACTACCGGTAGAACTTACGAGGTCGACTGGAGGTGAAGTATGAGAAAATTCCAAATCATCGTAGCAACGAGCGTAGAACTAAGAGCGGATAATGGGGATGACCTTCTCCAATTTGTAGACTAAAGAGGATGATCCAGGACAAGCAGCAGAGCAAGCGGCAGAGCTTGATCGGCGGAGCAAGCGGTAGAGCTAGAACGGTGGAACTTCAGTATCCAAGACGAAAGAGTATGATTCAGGACTACCGGTGGAGCA"
ob_sequnence="GMCKASTCRACCAYSCACTGGTMTCWTGAYSCRYGGTMAKGGARGCKAKTRTWAWGSKRCKAGCCGSCMGGGCACRGKCATMTRKCGATWCYATTGMCCYSCGCTYKTTGCMAKCAACYYMGCKCGWGSARTYRACRTWARGSGTTYSCGACTTTKKGKGTTKGCATATMCTAASAMMWAYGKATYMCMRTMSTCYSGCCCTYGCCKRAYSTTCCCKSRTKTACTGGWAYCGATMCAYSCCCCYKTGTAMAKCAAGYYMGAKTAWCSARAYRCCRTWGSGRAYRTGRAGWTGWTRTYRTCRGWGSTRCYRTCRACWTCWARTYRTTRTWGMCKYMATATTAGYKTTTTMAKCGCTYYMTAKGAWTSARCYRAGRAW"
encoded_sequnence="TMAKCSGTRGGATYSAGGACTGMCAWATGYSGRYACCMGKATGRAAKTKARAWCWTSKRAKCAAGTSAMAATGGTRAKACCMGRKGACCWTYTCCAMTTYSTAGAYKAAAGMGKATGAYYMAGKACWASCRGYRGARCWTRCSAGGYSGACTGGAKKTKAAGKATGAGAMAATTSCMMWTYAKCGYMGMRAMSAGYSTAGAAYTAAKRGYSGATAAKSRGKATGACCWTYTCCAMTTYSTAGAYKAAAGMGKATGAYYMAGKACWASCRGYRGARCWASCRGYRGARCTWGAWCRGYRGARCWASCRGYRGARCTWGAWCRGYRGARCWTMAKYMTCCAAGAYKAAAGMGKATGAYYMAGKACWASCRGYRGARCW"
ref="KMMKMSKYRRSMWYSMRSWSKKMYMWWKRYSSRYRSYMRKRKRRRMKWKWRWWMWKSKRMKMRMSKSMMRRKSRYRRKMMYMKRKSRMYWYYWYYRMYYYSYRSWYKWWRSMRKMWRMYYMRSKMSWRSMRKYRRMRYWWRSSRKKYSSRMYKKWKKKKRWKKRYRWRWMMWWWSMMMWWYRKMKYMSMRWMSWSYSKMSMWYKMMKRRYSKWYMMKSRKKWWSWSSWWYYSMWMYWYSYMSMYKWRWRMRKMWRRYYMRRKWMWMSMRRYRSMRYWRSSRRYRKRRMKWKRWYRKYRKMRSWRSYRSYRKMRMYWKMWMRKYRKWRYWKMMKYMWYMWWRRYKWWWKMRKMKSWYYMWRKRMWWSMRSYRRRRMW"

# Sequencing analyze 

In [15]:
def find_overlap(seq1, seq2):
    """Find the longest overlap between the end of seq1 and the start of seq2."""
    overlap_length = 0
    for i in range(1, min(len(seq1), len(seq2)) + 1):
        if seq1[-i:] == seq2[:i]:
            overlap_length = i
    return overlap_length

def combine_sequences(seq1, seq2):
    """Combine two sequences by removing the overlapping part from the second sequence."""
    overlap_length = find_overlap(seq1, seq2)
    return seq1 + seq2[overlap_length:]
def organize_reads(input_file, output_file):
    with open(input_file, 'r') as file:
        sequences = [line.strip() for line in file.readlines()]
    for i in range(len(sequences)):
        if ' ' not in sequences[i]:
            continue
        seq1,seq2=sequences[i].split(' ')
        combined_sequence = combine_sequences(seq1, seq2)
        sequences[i]=combined_sequence
    with open(output_file, 'w') as file:
        for sequence in sequences:
            file.write(sequence + '\n')
def extract_sequences(input_file, output_file):
    target_start = 'GCGATCGC'
    target_end = 'GAATTCTGCAGTCGACGGTAC'
    
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if target_start in line and target_end in line:
                parts = line.split(target_start)
                if len(parts) > 2:
                    # Extract between second occurrence of start and first occurrence of end
                    sub_part = target_start.join(parts[2:])  # Get the part after the second GCGATCGC
                    seq = sub_part.split(target_end)[0]  # Get up to first GAATTCT...
                else:
                    # If there's only one occurrence of GCGATCGC
                    sub_part = parts[1]  # Get the part after the first GCGATCGC
                    seq = sub_part.split(target_end)[0]  # Get up to first GAATTCT...
                outfile.write(seq + '\n')
            elif target_start in line:
                # Only the start is present
                sub_part = line.split(target_start)[1]
                seq = sub_part.split(target_end)[0]
                outfile.write(seq + '\n')
def extract_lines(input_file, output_file):
    lines = []
    with open(input_file, 'r') as file:
        for line in file:
            if len(line.strip()) == 376:
                lines.append(line)
    with open(output_file, 'w') as file:
        for line in lines:
            file.write(line)

In [ ]:
def read_fasta(filename):
    """Reads sequences from a FASTA file."""
    sequences = []
    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequences.append(line.strip())
    return sequences

def calculate_base_frequencies(sequences):
    """Calculates the frequency of each base at every position."""
    sequence_length = len(sequences[0])
    base_counts = [Counter() for _ in range(sequence_length)]

    for seq in sequences:
        for i, base in enumerate(seq):
            base_counts[i][base] += 1

    base_frequencies = []
    for counts in base_counts:
        total = sum(counts.values())
        frequencies = {base: count / total for base, count in counts.items()}
        base_frequencies.append(frequencies)
    return base_frequencies

def apply_cutoff(base_frequencies):
    """Keeps only the top 2 most frequent bases at each position."""
    filtered = []
    for freq in base_frequencies:
        # Sort bases by frequency (descending), keep top 2
        top2 = dict(sorted(freq.items(), key=lambda x: x[1], reverse=True)[:2])
        filtered.append(top2)
    return filtered

def replace_with_combined_bases(frequencies):
    """Replaces bases with IUPAC combined bases according to the frequencies."""
    iupac_codes = {
        frozenset(['A', 'G']): 'R', frozenset(['C', 'T']): 'Y',
        frozenset(['G', 'T']): 'K', frozenset(['A', 'C']): 'M',
        frozenset(['G', 'C']): 'S', frozenset(['A', 'T']): 'W',
        frozenset(['A']): 'A', frozenset(['C']): 'C',
        frozenset(['G']): 'G', frozenset(['T']): 'T',
    }

    consensus_sequence = ''
    for pos_freq in frequencies:
        significant_bases = frozenset(pos_freq.keys())
        consensus_sequence += iupac_codes.get(significant_bases, 'A')  # 'N' if no significant bases or combination not in table

    return consensus_sequence
def decryption  (consensus_sequence,obfuscation_seqs):
    combined_bases = {'R': ['A', 'G'], 'Y': ['C', 'T'], 'K': ['G', 'T'], 'M': ['A', 'C'], 'S': ['C', 'G'], 'W': ['A', 'T']}
    decrypted=''
    for i in range(len(consensus_sequence)):
        if consensus_sequence[i] == obfuscation_seqs[i]:
            decrypted+=consensus_sequence[i]
        else:
            choice=[b for b in combined_bases[consensus_sequence[i]] if b != obfuscation_seqs[i]]
            decrypted+=choice[0]
    return decrypted

In [17]:
import os
from os import listdir
from os.path import isfile, join
files = [f'/home/zzk/work/DNA_storage/30-1089119466/Step1/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step1/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step1/', f)) and f.endswith('.fastq')]
for file in files:
   os.system(f'awk \'NR%4 ==2\' {file} > /home/zzk/work/DNA_storage/30-1089119466//Step2/{file[file.rfind("/")+1:]}')  # Extract the sequences from the fastq files


In [18]:
files = [f'/home/zzk/work/DNA_storage/30-1089119466/Step2/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step2/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step2/', f)) and f.find("R2")!=-1]
for file in files:
    print(file)
    os.system(f'awk \'{{print $0}}\' {file} | rev | tr \'ATCGN\' \'TAGCN\' > /home/zzk/work/DNA_storage/30-1089119466/Step3/{file[file.rfind("/")+1:]}')  # reverse complement the R2 sequences


/home/zzk/work/DNA_storage/30-1089119466/Step2/P2_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P6_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P3_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P10_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P9_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P5_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P7_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P1_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P8_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P4_R2_001.fastq


In [19]:
files = [f'/home/zzk/work/DNA_storage/30-1089119466/Step2/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step2/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step2/', f)) and f.find("R1") != -1]
for file in files:
    file1 = file
    file2 = file.replace("Step2","Step3").replace("R1", "R2")
    print(file1, file2)
    os.system(f'paste -d " " {file1} {file2} > /home/zzk/work/DNA_storage/30-1089119466/Step4/{os.path.basename(file)}')


/home/zzk/work/DNA_storage/30-1089119466/Step2/P10_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P10_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P3_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P3_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P4_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P4_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P6_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P6_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P5_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P5_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P9_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P9_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P7_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P7_R2_001.fastq
/home/zzk/work/DNA_storage/30-1089119466/Step2/P1_R1_001.fastq /home/zzk/work/DNA_storage/30-1089119466/Step3/P1_R2_

In [20]:
files = [f'/home/zzk/work/DNA_storage/30-1089119466/Step4/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step4/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step4/', f)) and f.endswith('.fastq')]
for file in files:
    os.system(f'grep -E \".*AGCTGGGACCACCTTATATTCCCAG.*GAATTCTGCAGTCGACGGTACC.*\" {file} > /home/zzk/work/DNA_storage/30-1089119466/Step5/{file[file.rfind("/")+1:]}')  # filter the sequences with the primers


In [21]:
files=[f'/home/zzk/work/DNA_storage/30-1089119466/Step5/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step5/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step5/', f)) and f.endswith('.fastq')]
for file in files:
    output=file.replace("Step5","Step6")
    organize_reads(file,output) # combine the sequences without the overlapping part


In [22]:
files=[f'/home/zzk/work/DNA_storage/30-1089119466/Step6/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step6/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step6/', f)) and f.endswith('.fastq')]
for file in files:
    output=file.replace("Step6","Step7")
    extract_sequences(file,output) # extract the sequences between the primers
files=[f'/home/zzk/work/DNA_storage/30-1089119466/Step7/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step7/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step7/', f)) and f.endswith('.fastq')]
for file in files:
    output=file.replace("Step7","Step8")
    extract_lines(file,output) # extract the sequences with the length of 376, the length which was determined by the obfuscation data                    


In [27]:
import os
import csv
from collections import Counter
files=[f'/home/zzk/work/DNA_storage/30-1089119466/Step8/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step8/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step8/', f)) and f.endswith('.fastq')]
output_dir='/home/zzk/work/DNA_storage/30-1089119466/base_frequnceies/'
for file in files:
    sequences = read_fasta(file)
    base_frequencies = calculate_base_frequencies(sequences)
    # save the base frequencies the first 10 bases into a csv file and each base is one column
    csv_path = os.path.join(output_dir, os.path.basename(file).replace('.fastq', '.csv'))
    with open(csv_path, mode='w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Base'] + list(range(1, 377)))
        for base in ['A', 'C', 'G', 'T']:
            frequencies = [freq.get(base, 0) for freq in base_frequencies[:376]]
            writer.writerow([base] + frequencies)
    filtered_frequencies = apply_cutoff(base_frequencies)
    consensus_sequence = replace_with_combined_bases(filtered_frequencies)
    print(os.path.basename(file), consensus_sequence == ref)

P10_R1_001.fastq True
P3_R1_001.fastq True
P4_R1_001.fastq True
P6_R1_001.fastq True
P5_R1_001.fastq True
P9_R1_001.fastq True
P0_R1_001.fastq True
P7_R1_001.fastq True
P1_R1_001.fastq True
P2_R1_001.fastq True
P8_R1_001.fastq True


# Simulation

In [ ]:
import os 
import glob
from os import listdir
from os.path import isfile
from os.path import join
import numpy as np

files=[f'/home/zzk/work/DNA_storage/30-1089119466/Step8/{f}' for f in listdir('/home/zzk/work/DNA_storage/30-1089119466/Step8/') if isfile(join('/home/zzk/work/DNA_storage/30-1089119466/Step8/', f)) and f.endswith('.fastq')]
number_of_reads=[50,100,500,1000,5000,10000,50000]
# sort the files
files=sorted(files)
for file in files:
    i=os.path.basename(file).replace('_R1_001.fastq','')
    # shuffle the sequences in the file
    sequences=read_fasta(file)
    for m in range(0,100):
        np.random.shuffle(sequences)
        #get the first  number_of_reads[i] sequences and save them to a new file
        for n in number_of_reads:
            with open(f'/home/zzk/work/DNA_storage/Subsampling/{i}/{n}reads/S{m+1}', 'w') as f:
                for seq in sequences[:n]:
                    f.write(seq+'\n')
                    